In [1]:
import os
import os.path as op
from glob import glob
import numpy as np
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
# import matplotlib.pyplot as plt
import cv2

_y_max_parking = 0.66
frames_to_get_result = 20
time_to_dif_pos = 5
frames_to_extend_result = 20*3
threshold = 0.05

In [2]:
def extract_subclip(path_txt,
                   path_video,
                   folder_save_video):
    car = {}
    max_frame = 0


    with open(path_txt) as f:
        for line in f.readlines():
            line = line.split()
            frame = int(line[0])
            if frame > max_frame:
                max_frame = frame
            idx = line[1]
            left = int(line[2])
            top = int(line[3])
            w = int(line[4])
            h = int(line[5])
            W = int(line[6])
            H = int(line[7])
            _x = (left+w/2)/W
            _y = (top+h/2)/H
            _w = w/W
            _h = h/H
            try:
                car[idx]['position'][frame] = [_x,_y,_w,_h]
                if _y+_h/2 > _y_max_parking:
                    car[idx]['parking'] = True
                if (frame-frames_to_get_result*time_to_dif_pos) in car[idx]['position']:
                    _x_acc = _x - car[idx]['position'][frame-frames_to_get_result*time_to_dif_pos][0]
                    _y_acc = _y - car[idx]['position'][frame-frames_to_get_result*time_to_dif_pos][1]
                    car[idx]['pos_dif'][int(frame-frames_to_get_result/2)] = np.sqrt(_x_acc**2+_y_acc**2)
            except Exception as e:
                print(e)
                car[idx] = {}
                car[idx]['position'] = {}
                car[idx]['position'][frame] = [_x,_y,_w,_h]
                car[idx]['parking'] = False
                car[idx]['pos_dif'] = {}
                
    for idx in car:
        if not car[idx]['parking']:
            continue
        # print(f'car {idx}')
        # time = []
        # acc = []
        # for i in car[idx]['pos_dif']:
        #     time.append(i)
        #     acc.append(car[idx]['pos_dif'][i])
        # plt.plot(time,acc)
    
        check_frames = []
        for frame in car[idx]['pos_dif']:
            before_acc = 0
            after_acc = 0
            if int(frame-frames_to_get_result) in car[idx]['pos_dif']:
                before_acc = car[idx]['pos_dif'][int(frame-frames_to_get_result)]
            if int(frame+frames_to_get_result) in car[idx]['pos_dif']:
                after_acc = car[idx]['pos_dif'][int(frame+frames_to_get_result)]
            current_acc = car[idx]['pos_dif'][frame]
            if (before_acc>threshold) and (after_acc>threshold) and (current_acc>threshold):
                check_frames.append(frame)
        if len(check_frames) == 0:
            continue
        start = max(min(check_frames)-frames_to_get_result*time_to_dif_pos-frames_to_extend_result,0)
        stop = min(max(check_frames)+frames_to_get_result*time_to_dif_pos+frames_to_extend_result,max_frame)
        video = cv2.VideoCapture(path_video)
        frames_per_second = video.get(cv2.CAP_PROP_FPS)
        i = 1
        os.makedirs(folder_save_video,exist_ok=True)
        while True:
            basename_video = op.basename(path_txt.replace('.txt',f'_{i}.mp4'))
            path_save_video = op.join(folder_save_video,basename_video)
            if not op.exists(path_save_video):
                print(f'Cut video {path_video}')
                print(f'Save video to {path_save_video}')
                ffmpeg_extract_subclip(path_video, 
                                       start/frames_per_second, 
                                       stop/frames_per_second, 
                                       targetname=path_save_video)
                # print(start/frames_per_second)
                # print(stop/frames_per_second)
                break
            else:
                i += 1

In [ ]:
sources = '/data/results/*'
paths = glob(sources)
while len(paths) != 0:
    for path in paths:
        if path[-4:] != '.txt':
            continue
        # print(path)
        extract_subclip(path_txt=path,
                       path_video=path.replace('results','extracted_videos').replace('txt','mp4'),
                       folder_save_video=op.dirname(path.replace('results','results2')))
    sources = op.join(sources,'*')
    paths = glob(sources)